Here I'm going to basically try to predict the missing values in the "Market Category" column

In [12]:
import numpy as np
import pandas as pd
from numpy import mean, std
# from sklearn.preprocessing import OneHotEncoder
# from sklearn.model_selection import RepeatedKFold
# from keras.models import Sequential
# from keras.layers import Dense
# from sklearn.metrics import accuracy_score

In [18]:
df = pd.read_csv('data.csv')

In [4]:
df['Engine Fuel Type'].fillna(method='ffill', inplace=True)
df['Engine HP'].fillna(df['Engine HP'].median(), inplace = True)
df['Engine Cylinders'].fillna(df['Engine Cylinders'].mean(), inplace = True)
df['Number of Doors'].fillna(method='ffill', inplace = True)
df.drop('Model', axis=1, inplace=True)

In [ ]:
#we need to drop the model of the car
#to OHE categorical columns

# categorical_columns = ['Make', 'Engine Fuel Type', 'Transmission Type', 'Driven_Wheels', 'Vehicle Size', 'Vehicle Style']

# #encoding of categorical columns
# for column in categorical_columns:
#     tempdf = pd.get_dummies(df[column],  prefix=column)
#     df = pd.concat([df, tempdf], axis=1)

#     df.drop(column, axis=1, inplace=True)

#WE DO NOT REALLY NEED TO ENCODE SINCE WEKA WOULD TAKE CARE OF THIS ISSUE

In [23]:
#df['Market Category'].unique()
y = df['Market Category']
y.unique()

array(['Factory Tuner,Luxury,High-Performance', 'Luxury,Performance',
       'Luxury,High-Performance', 'Luxury', 'Performance', 'Flex Fuel',
       'Flex Fuel,Performance', nan, 'Hatchback',
       'Hatchback,Luxury,Performance', 'Hatchback,Luxury',
       'Luxury,High-Performance,Hybrid', 'Diesel,Luxury',
       'Hatchback,Performance', 'Hatchback,Factory Tuner,Performance',
       'High-Performance', 'Factory Tuner,High-Performance',
       'Exotic,High-Performance', 'Exotic,Factory Tuner,High-Performance',
       'Factory Tuner,Performance', 'Crossover', 'Exotic,Luxury',
       'Exotic,Luxury,High-Performance', 'Exotic,Luxury,Performance',
       'Factory Tuner,Luxury,Performance', 'Flex Fuel,Luxury',
       'Crossover,Luxury', 'Hatchback,Factory Tuner,Luxury,Performance',
       'Crossover,Hatchback', 'Hybrid', 'Luxury,Performance,Hybrid',
       'Crossover,Luxury,Performance,Hybrid',
       'Crossover,Luxury,Performance',
       'Exotic,Factory Tuner,Luxury,High-Performance',
   

In [5]:
#Market Category
#this is a categorical column
lst = df['Market Category'].unique()

category_list = []
#loop extracts unique label for the market category column 
for i in lst:
    if isinstance(i, str):
        if ',' in i:
            new_list = i.split(',')
            for j in new_list:
                if j in category_list:
                    print(j, 'already found')
                    continue
                else:
                    category_list.append(j)
                    print(j, 'added')
        else:
            if i in category_list:
                print(i, 'already found')
                continue
            else:
                category_list.append(i)
                print(i, 'added')


Factory Tuner added
Luxury added
High-Performance added
Luxury already found
Performance added
Luxury already found
High-Performance already found
Luxury already found
Performance already found
Flex Fuel added
Flex Fuel already found
Performance already found
Hatchback added
Hatchback already found
Luxury already found
Performance already found
Hatchback already found
Luxury already found
Luxury already found
High-Performance already found
Hybrid added
Diesel added
Luxury already found
Hatchback already found
Performance already found
Hatchback already found
Factory Tuner already found
Performance already found
High-Performance already found
Factory Tuner already found
High-Performance already found
Exotic added
High-Performance already found
Exotic already found
Factory Tuner already found
High-Performance already found
Factory Tuner already found
Performance already found
Crossover added
Exotic already found
Luxury already found
Exotic already found
Luxury already found
High-Performa

In [6]:
#create new df containg rows with missing market category (test_df)
prediction_df = df.loc[df['Market Category'].isna()]
#drop rows with missing market category
df.drop(df.index[df['Market Category'].isna()], inplace=True)
#drop market category column
prediction_df.drop('Market Category', axis=1, inplace=True)

/tmp/ipykernel_160867/3728987043.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prediction_df.drop('Market Category', axis=1, inplace=True)


In [15]:
#output column ('y')
df_y = df.pop('Market Category')
#changing format of column rows into one that is easier to encodes
df_y = df_y.apply(lambda x: [x])

In [16]:
print(type(df['Engine HP']))
print(type(df_y))
print(df_y.unique())

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


TypeError: unhashable type: 'list'

In [ ]:
df = df.reset_index()
#df_y = df_y.reset_index()

In [ ]:
#custom OHE function for y_column

def ohe_list(df_col, categories):
    #create a dictionary map
    label_to_int = dict((c, i) for i, c in enumerate(categories))
    #int_to_label = dict((i, c) for i, c in enumerate(categories))

    #encode to integer
    label_encoded = [[[label_to_int[label] for label in cell.split(',') ] for cell in row] for row in df_col]

    #create one hot list
    oh_list = list()

    for row in label_encoded:
        for cell in row:
            cell_enc = [0 for _ in range(len(categories))]
            for label in cell:
                cell_enc[label] = 1
            oh_list.append(cell_enc)

    return oh_list 


In [ ]:
df_y_enc = ohe_list(df_y, category_list)

In [ ]:
df_y_enc = pd.DataFrame(df_y_enc)
df_y_enc

In [ ]:
def get_model(n_inputs, n_outputs):
	model = Sequential()
	model.add(Dense(20, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
	model.add(Dense(n_outputs, activation='sigmoid'))
	model.compile(loss='binary_crossentropy', optimizer='adam')
	return model

#really lame model lol

In [ ]:
def evaluate_model(X, y):
    results = list()
    n_inputs, n_outputs = X.shape[1], y.shape[1]

    #defining evaluation precedure
    cross_val = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
    for train, test in cross_val.split(X):
        X_train, X_test = X.loc[train, X.columns], X.loc[test, X.columns]
        y_train, y_test = y.loc[train, y.columns], y.loc[test, y.columns]

        model = get_model(n_inputs, n_outputs)
        model_history = model.fit(X_train, y_train, epochs=10)

        y_hat = model.predict(X_test)
        y_hat = y_hat.round()

        acc = accuracy_score(y_test, y_hat)
        print('\n >%.3f \n\n' %acc)
        results.append(acc)
    return results, model_history
        
#well this was pointless

In [ ]:
#results, model_hist = evaluate_model(df, df_y_enc)
#print('Accuracy: %.3f (%.3f)' % (mean(results), std(results)))

#avg accuracy is 30% (really poor)

Create new data set to use on weka 

In [ ]:
df['y_axis'] = df_y_enc[df_y_enc.columns[0:]].apply(lambda x: [i for i in x], axis=1)

In [ ]:
df.to_csv('processed_data.csv', index=False)

In [ ]:
test_df = pd.read_csv('to_predict.csv')  #this csv file was deleted
test_df.columns

In [ ]:
#test_df.drop(test_df['index'], axis=1, inplace=True)
del test_df['index']
test_df.reset_index(inplace=True)
test_df['y_axis'] = '?'

In [ ]:
test_df.to_csv('to_pred.csv', index=False, index_label=False)

### NOTES ###
-> Weka can only predice a certain amount of supplied test values at a time (Well at least I have'nt found a way around it)  

-> The test dataset (to_predict) must have the same columns as the train dataset when suppling to weka (including the column with missing values... all you have to do is to populate that column with "?")  

-> Javabridge installation is not working (I have to find a way to fix this asap!)  

-> Decode function not yet imlpemented (This would be completed later)


In [24]:
a:set = ()
a.add(1)
print(a)

AttributeError: 'tuple' object has no attribute 'add'